# SLR Automation

The systamatic lireature review (SLR) is one of widely used methods to combine & sythesize novel insights from divese litreature.

However, there are various tasks of SLR which researcher have to manually. These tasks include following

1. Combining Data Scopus and Web of Sciennce Data
2. Removing Duplicates
3. Removing Articles which should not be reviewed due to lack of scope on area being studies. 

In this, code book, we automate the above tasks 

# 1. Let's load the data.

The scopus data normally is the csv format but web of science data is in excel format.

It is suggested that, we should also convert web of science data into csv forma. 
We can easily do by using save as function and choosing format Comma Delimited csv. 

In [4]:
# Lets load the web of science data after convertibng it into the csv. 
# we have used encoding= latin-1 due to change from excel to csv.
import pandas as pd
web_of_science_df = pd.read_csv('savedrecs.csv', encoding='latin-1')

In [5]:
# it is always nice to see the columns we have in the csv data.
web_of_science_df.columns

Index(['Publication Type', 'Authors', 'Book Authors', 'Book Editors',
       'Book Group Authors', 'Author Full Names', 'Book Author Full Names',
       'Group Authors', 'Article Title', 'Source Title', 'Book Series Title',
       'Book Series Subtitle', 'Language', 'Document Type', 'Conference Title',
       'Conference Date', 'Conference Location', 'Conference Sponsor',
       'Conference Host', 'Author Keywords', 'Keywords Plus', 'Abstract',
       'Addresses', 'Affiliations', 'Reprint Addresses', 'Email Addresses',
       'Researcher Ids', 'ORCIDs', 'Funding Orgs', 'Funding Name Preferred',
       'Funding Text', 'Cited References', 'Cited Reference Count',
       'Times Cited, WoS Core', 'Times Cited, All Databases',
       '180 Day Usage Count', 'Since 2013 Usage Count', 'Publisher',
       'Publisher City', 'Publisher Address', 'ISSN', 'eISSN', 'ISBN',
       'Journal Abbreviation', 'Journal ISO Abbreviation', 'Publication Date',
       'Publication Year', 'Volume', 'Issue', 'Pa

In [6]:
# Now we can should also load the scopus data as well.
scopus_df = pd.read_csv('scopus.csv', encoding='latin-1')

In [7]:
# let also check its columns.
scopus_df.columns

Index(['ï»¿"Authors"', 'Author full names', 'Author(s) ID', 'Title', 'Year',
       'Source title', 'Volume', 'Issue', 'Art. No.', 'Page start', 'Page end',
       'Page count', 'Cited by', 'DOI', 'Link', 'Affiliations',
       'Authors with affiliations', 'Abstract', 'Author Keywords',
       'Index Keywords', 'Molecular Sequence Numbers', 'Chemicals/CAS',
       'Tradenames', 'Manufacturers', 'Funding Details', 'Funding Texts',
       'References', 'Correspondence Address', 'Editors', 'Publisher',
       'Sponsors', 'Conference name', 'Conference date', 'Conference location',
       'Conference code', 'ISSN', 'ISBN', 'CODEN', 'PubMed ID',
       'Language of Original Document', 'Abbreviated Source Title',
       'Document Type', 'Publication Stage', 'Open Access', 'Source', 'EID'],
      dtype='object')

# Data Targeted and Handle

Now
1. we will combine them. But, We need to keep in mind that, names of columns are different.
2. we need to combine them in a way that, we retain the articles by using thier title. 
3. After combining the articles, we will remove the duplicates.  

# 1 Let's Combine 

In [21]:
# So, let combine and remove duplicates

import pandas as pd

# 1. Load the data agaig. No need to load but its ok load
scopus_df = pd.read_csv('scopus.csv', encoding='latin-1')
web_of_science_df = pd.read_csv('savedrecs.csv', encoding='latin-1')

# 2. It is highly necessary to Convert titles titles to lowercase for removing duplicates
scopus_df['Title'] = scopus_df['Title'].str.lower()
web_of_science_df['Article Title'] = web_of_science_df['Article Title'].str.lower()

# 3. Create new dataframes which only conatins 'Title' and 'Abstract' columns.
     #We can retain the ret of data but not necessary for now. 
scopus_title_abstract_df = scopus_df[['Title', 'Abstract']]
web_of_science_title_abstract_df = web_of_science_df[['Article Title', 'Abstract']]

# 3. Rename columns for consistency
    # As we know that, in web of science name of column of Title is Article Title, so we need to rename it.  
web_of_science_title_abstract_df = web_of_science_title_abstract_df.rename(columns={'Article Title': 'Title'})

# 4. Now, Let Concatenate or merge both dataframes
combined_title_abstract_df = pd.concat([scopus_title_abstract_df, web_of_science_title_abstract_df], ignore_index=True)

# Save the combined data to a new CSV file
combined_title_abstract_df.to_csv('combined_title_abstract_data.csv', index=False, encoding='utf-8')


# 2. Let's Remove Duplicates

In [22]:
import pandas as pd

# 1. Load the combined data 
combined_title_abstract_df = pd.read_csv('combined_title_abstract_data.csv', encoding='utf-8')

# 2. Count the number of duplicates before removal
total_duplicates_before = combined_title_abstract_df.duplicated(subset='Title').sum()

# 3. Convert titles to lowercase and remove duplicates based on the 'Title' column
    #Although it is handeled but it is good to have this again. 
deduplicated_combined_df = combined_title_abstract_df.drop_duplicates(subset='Title', keep=False)

# 4. Count the number of duplicates after removal
total_duplicates_after = deduplicated_combined_df.duplicated(subset='Title').sum()

# 5. Save the deduplicated data to a new CSV file
deduplicated_combined_df.to_csv('deduplicated_combined_title_abstract_data.csv', index=False, encoding='utf-8')

# Print the number of duplicates before and after removal
print(f"Number of duplicates before removal: {total_duplicates_before}")
print(f"Number of duplicates after removal: {total_duplicates_after}")


Number of duplicates before removal: 68
Number of duplicates after removal: 0


# Removing article which does not add any Value:
    
So, when we are searching into database. we will get the raw data.

However, this data of research articles does contains articles which does not add value in answering the research questions being targered. 

So, Researcher have to remove them.

One of widely used practicse is to read the abstract of each article and identify each article's relavance in terms of research question being developed. 

So, one of the way is to see whether in the abstract is discussing the key terms. 

For Example: Our research artilce is reviewing litreature systamatically on Project Management and Ambidextrious innovation.

It means in abstract, we should be able to find key term of Ambidexrious Innovation discussed along with key terminology of Project managmenet. 

So, here we will have the develop function which will go into abstract and locate whether key terms of ambidextrious innovation id discussedwith the project managment or not. 

So, paper which discussed will be labelled as yes and paper which does not discuss will be labeled as No.



In [28]:
# 1. Load the data. 

import pandas as pd

deduplicated_combined_df = pd.read_csv('deduplicated_combined_title_abstract_data.csv', encoding='utf-8')

# Keywords set for both
# Alwats keep keywords in small case as abstract contains these into small case.
# Be sure to set keywords. May be work on it little bit.   
first_set_keywords = ["ambidextrous Innovation", "innovation", "ambidextrous Innovation", "ambidexterity", "exploration", "exploitation", "exploitative innovation", "explorative innovation"]
second_set_keywords = ["project","projects" "project management", "project office"]

# Function to check if abstract contains at least one word from both sets of keywords
def label_abstract(row):
    abstract = row['Abstract']
    
    if pd.notna(abstract): 
        first_set_found = any(keyword in abstract for keyword in first_set_keywords)
        second_set_found = any(keyword in abstract for keyword in second_set_keywords)
        return 'Yes' if first_set_found and second_set_found else 'No'
    else:
        return 'No'  # Assuming that if abstract is NaN, then label should be 'No'

# Apply the function to create a new 'Label' column
deduplicated_combined_df['Label'] = deduplicated_combined_df.apply(label_abstract, axis=1)

# Save the updated dataframe to a new CSV file
deduplicated_combined_df.to_csv('labeled_data.csv', index=False, encoding='utf-8')


In [34]:
# Let's Count the Yes and No

import pandas as pd
#1. Load the data
labeled_df = pd.read_csv('labeled_data.csv', encoding='utf-8')

# 2. Count the number of 'Yes' and 'No' labels
count_yes = (labeled_df['Label'] == 'Yes').sum()
count_no = (labeled_df['Label'] == 'No').sum()

# 3. Print the counts
print(f"Number of 'Yes' labels: {count_yes}")
print(f"Number of 'No' labels: {count_no}")


Number of 'Yes' labels: 99
Number of 'No' labels: 148


In [33]:
import pandas as pd

# 1. Assuming 'labeled_data.csv' is the name of the CSV file
labeled_df = pd.read_csv('labeled_data.csv', encoding='utf-8')

# 2. Filter rows with 'Yes' labels
yes_df = labeled_df[labeled_df['Label'] == 'Yes']

# 3. Save the 'Yes' labeled data to a new CSV file
yes_df.to_csv('yes_labeled_data.csv', index=False, encoding='utf-8')